# Random matrix product state calculations v2

Updated to use TensorNetwork library and tighter distribution 

In [4]:
import numpy as np
import math
import tensornetwork as tn
from scipy.stats import unitary_group as ug

In [15]:
def gue(d):
    #TODO: check normalization
    A = np.random.normal(size=(n,m,m,d)) + 1j*np.random.normal(size=(n,m,m,d))
    return 2**(-d**0.5 * 0.5)*math.pi**(-d*0.5)*(A + np.conjugate(A.T))

def random_mps(n,d,m):
    """
    Returns a random matrix product state of size n, local dimension d, and bipartite Schmidt rank m as a ndarray object.
    List index identifies the qudits, node axes 0 and 1 are left and right Schmidt indices, and axis 2 is qudit indices.
    """
#     if (m**n > (d**n)*(d**n + 1)/2):
#         print("[-] Representation can be reduced")
#     if (m**n > (d**n)**2):
#         print("[!] Representation not efficient")
#     return np.random.normal(size=(n,m,m,d)) + 1j*np.random.normal(size=(n,m,m,d))
    result = [tn.Node(np.random.normal(size=(m,m,d)) + 1j*np.random.normal(size=(m,m,d))) for _ in range(n)]
    return result, [result[i][1]^result[i+1][0] for i in range(n-1)]

def random_left_condition(m):
    """
    Returns a random left boundary condition. Axis 0 required for reshaping, axis 1 is Schmidt contraction indices.
    """
    return tn.Node(np.random.normal(size=(m)) + 1j*np.random.normal(size=(m)))

def random_right_condition(m):
    """
    Returns a random right boundary condition. Axis 0 is Schmidt contraction indices, axis 1 required for reshaping.
    """
    return tn.Node(np.random.normal(size=(m)) + 1j*np.random.normal(size=(m)))

def random_operator(n,d):
    U = ug.rvs(d**n)
    return tn.Node(np.dot(np.conjugate(U).T,np.dot(np.diag(np.random.normal(size=d**n)),U)))

In [39]:
n = 4
d = 2
m = 3
s,e = random_mps(n,d,m)
l = random_left_condition(m)
r = random_right_condition(m)
O = random_operator(1,d)

In [40]:
s[0] = tn.contract(l[0]^s[0][0])
for i in range(n-1):
    s[i+1] = tn.contract(e[i])
    s[i] = 1
s[n-1] = tn.contract(s[n-1][-2]^r[0])
conj = tn.Node(tn.conj(s[n-1].get_tensor()))
for i in range(n):
    conj[i] ^ s[n-1][i]
print((s[n-1] @ conj).get_tensor())

(31718.217018365955+0j)
